In [ ]:
%matplotlib inline
%load_ext rpy2.ipython
import vdmlab as vdm
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from core import assign_label, Rat, combine_rats
from plotting import plot_behavior

from rpy2 import robjects as ro
from rpy2.robjects import pandas2ri
from rpy2.robjects.packages import importr

In [ ]:
pandas2ri.activate()
lme4 = importr('lme4')
car = importr('car')
lmerTest = importr('lmerTest')

In [ ]:
# home = os.path.expanduser("~")
# root = os.path.join(home, 'code', 'emi_biconditional')
# data_filepath = os.path.join(root, 'cache', 'data')

data_filepath = os.path.join('E:\\', 'code', 'emi_biconditional', 'cache', 'data')

In [ ]:
broken_sessions = ['!2016-10-19a1', '!2016-10-19a2']
sessions = []
for file in sorted(os.listdir(data_filepath)):
    if file[0] == '!' and file not in broken_sessions:
        sessions.append(os.path.join(data_filepath, file))

rats = ['1', '2', '3', '4', '5', '6', '7', '8']

data = dict()
for rat in rats:
    data[rat] = Rat(rat)

broken_a = os.path.join(data_filepath, broken_sessions[0])
broken_b = os.path.join(data_filepath, broken_sessions[1])
rats_data_a = vdm.load_medpc(broken_a, assign_label)
rats_data_b = vdm.load_medpc(broken_b, assign_label)
for rat in rats_data_a:
    for key in rats_data_a[rat]:
        rats_data_b[rat][key].join(rats_data_a[rat][key])
for rat in rats:
    data[rat].add_session(**rats_data_b[rat])

for session in sessions:
    rats_data = vdm.load_medpc(os.path.join(data_filepath, session), assign_label)

    for rat in rats:
        data[rat].add_session(**rats_data[rat])

n_sessions = len(data['1'].sessions)
only_sound = False

df = combine_rats(data, rats, n_sessions, only_sound)

In [ ]:
df.head()

In [ ]:
def evaluate_training(learning_df):
    ro.globalenv['rlearn'] = pandas2ri.py2ri(learning_df)
    ro.r("""rlearn <- within(rlearn, {
      trial <- factor(trial_type)
      rewarded <- factor(rewarded)
      measure <- factor(measure)
      value <- as.numeric(value)})""")

    ro.r("contrasts(rlearn$trial) <- c(3, 1, -1, -3)")
    ro.r("contrasts(rlearn$rewarded) <- c(1, -1)")

    ro.r("lmm <- lmer(value ~ rewarded * session + (1|measure), data=rlearn, REML=FALSE)")
    anova = ro.r("print(Anova(lmm,type=3))")
    summary = ro.r("print(summary(lmm))")
    return anova, summary

In [ ]:
stats_df = df[df['cue'] == 'sound']
stats_df.head()

In [ ]:
anova, summary = evaluate_training(stats_df)

In [ ]:
anova

In [ ]:
summary